In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
import keras_hub
from sklearn.model_selection import train_test_split
from keras import backend as K
# import evaluate
from tqdm.auto import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-09-18 13:00:31.729783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758200431.909853      26 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758200431.963607      26 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/__script__.py
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/typing_extensions-4.15.0-py3-none-any.whl
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/requests-2.32.5-py3-none-any.whl
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/pandas-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/aiohttp-3.12.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/evaluate-0.4.5-py3-none-any.whl
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/tqdm-4.67.1-py3-none-any.whl
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/__results__.html
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/dill-0.4.0-py3-none-any.whl
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/multiprocess-0.70.16-py311-none-any.whl
/kaggle/input/pm-91260771-at-09-18-2025-12-59-26/PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle

In [2]:
MODEL_NAME = "bert_small_en_uncased"
SEQUENCE_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 5e-6

In [3]:
train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

In [4]:
train_df

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [5]:
# Take the first prompt and its associated response
train_df["prompt"] = train_df.prompt.map(lambda x: eval(x)[0])
train_df["response_a"] = train_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
train_df["response_b"] = train_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

In [6]:
train_df

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"A simple mnemonic for π:\n""How I wish I could ...","Sure, let's break it down:\n\n1. ""How"" has 3 l...",Here is how that mnemonic represents the digit...,1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"In python, implement a naive Bayes with gaussi...",Here is an implementation of a naive Bayes cla...,Sure! Here's an implementation of a naive Baye...,1,0,0
57474,4294710549,claude-1,alpaca-13b,is it unethical to work on building weapons?,Working on weapons technology raises some ethi...,It depends on the context. Weapons can be used...,1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"If a bait contains 0,0025% bromadiolon then ho...",Bromadiolone is a rodenticide which is most of...,"As an AI language model, I do not promote or c...",0,1,0


In [7]:
class TextPreprocessor:
    def __init__(self):
        self.tokenizer = keras_hub.tokenizers.BertTokenizer.from_preset(MODEL_NAME)

    def clean_text(self, text):
        text = str(text)
        
        # Handle literal \n characters (escaped newlines in the text)
        text = text.replace('\\n', ' ')
        text = text.replace('\\r', ' ')
        text = text.replace('\\t', ' ')
        
        ### Light
        text = text.lower() # lowercase everything
        text = re.sub(r"\s+", " ", text)  # Collapse whitespace
        text = re.sub(r"[^\x00-\x7F]+", " ", text)  # Remove non-ASCII
    
        text = text.replace('\\', ' ')
        
        return text.strip()

    def create_input_pairs(self, row):
        prompt = self.clean_text(row['prompt'])
        return [
            f"Prompt: {prompt} {self.tokenizer.sep_token} Response: {self.clean_text(row['response_a'])}",
            f"Prompt: {prompt} {self.tokenizer.sep_token} Response: {self.clean_text(row['response_b'])}"
        ]

processor = TextPreprocessor()

I0000 00:00:1758200453.935370      26 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758200453.936072      26 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [8]:
# Apply preprocessing
train_df['inputs'] = train_df.apply(processor.create_input_pairs, axis=1)
test_df['inputs'] = test_df.apply(processor.create_input_pairs, axis=1)

# Create labels (0: model_a wins, 1: model_b wins, 2: tie)
train_df['label'] = train_df[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1)
train_df['label'] = train_df['label'].map({'winner_model_a': 0, 'winner_model_b': 1, 'winner_tie': 2})

# Preview processed data
train_df[['inputs', 'label']].head()

,inputs,label
0,[Prompt: is it morally right to try to have a ...,0
1,[Prompt: what is the difference between marria...,1
2,[Prompt: explain function calling. how would y...,2
3,[Prompt: how can i create a test set for a ver...,0
4,[Prompt: what is the best way to travel from t...,1


In [9]:
train_df[['inputs', 'label']].iloc[57475]["inputs"]

['Prompt: if a bait contains 0,0025% bromadiolon then how much an adult rat needs to consume to die within 24 hours? [SEP] Response: bromadiolone is a rodenticide which is most often used in bait. it is a second generation anticoagulant that blocks the production of vitamin k1, which is essential for blood clotting. without vitamin k1, the blood will not clot properly and the rat will bleed to death. bromadiolone is effective at killing rats in a matter of hours and is often used in areas where rats are a problem, such as around homes, businesses, and farms.',
 'Prompt: if a bait contains 0,0025% bromadiolon then how much an adult rat needs to consume to die within 24 hours? [SEP] Response: as an ai language model, i do not promote or condone the use of poison or harm to any living beings. however, i can provide you with some information based on the question you asked. bromadiolone is a highly toxic rodenticide, and the lethal dose required to kill an adult rat can vary depending on s

In [10]:
train_df, val_df = train_test_split(
    train_df, 
    test_size=0.1, 
    stratify=train_df['label'],
    random_state=1025
)

In [11]:
# Preprocessing function
def preprocess_pair(text_pair, label=None):
    """Convert raw text pairs to model-ready format"""
    # Tokenize each response separately
    processed_a = preprocessor(text_pair[0])
    processed_b = preprocessor(text_pair[1])

    # Stack to create (2, seq_len) tensors
    model_inputs = {
        "token_ids": tf.stack([processed_a["token_ids"], processed_b["token_ids"]], axis=0),
        "padding_mask": tf.stack([processed_a["padding_mask"], processed_b["padding_mask"]], axis=0),
        "segment_ids": tf.stack([processed_a["segment_ids"], processed_b["segment_ids"]], axis=0)
    }
    return (model_inputs, label) if label is not None else model_inputs

def create_dataset(text_pairs, labels=None, preprocessor=None):
    """Convert to optimized TF Dataset with proper input pair handling"""
    AUTO = tf.data.AUTOTUNE

    # Convert to TensorFlow Dataset
    if labels is not None:
        ds = tf.data.Dataset.from_tensor_slices((text_pairs, labels))
        ds = ds.shuffle(1025)
    else:
        ds = tf.data.Dataset.from_tensor_slices(text_pairs)

    # Apply preprocessing and batching
    ds = ds.map(preprocess_pair, num_parallel_calls=AUTO)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTO)
    return ds

In [12]:
# Initialize preprocessor
preprocessor = keras_hub.models.TextClassifierPreprocessor.from_preset(
    MODEL_NAME,
    sequence_length=SEQUENCE_LENGTH
)

In [13]:
# Prepare all datasets
train_ds = create_dataset(
    train_df['inputs'].tolist(), 
    tf.keras.utils.to_categorical(train_df['label']),
    preprocessor=preprocessor
)
val_ds = create_dataset(
    val_df['inputs'].tolist(), 
    tf.keras.utils.to_categorical(val_df['label']),
    preprocessor=preprocessor
)
test_ds = create_dataset(
    test_df['inputs'].tolist(),
    preprocessor=preprocessor
)

In [14]:
for batch in train_ds.take(1):
    inputs, labels = batch
    print("Token IDs shape:", inputs["token_ids"].shape)  # Should be (batch_size, 2, 128)
    print("Mask shape:", inputs["padding_mask"].shape)
    print("Segment IDs shape:", inputs["segment_ids"].shape)
    print("Example token_ids[0,0,:5]:", inputs["token_ids"][0,0,:5], inputs["segment_ids"][0,0,:5])  # First 5 tokens of response_a

Token IDs shape: (16, 2, 128)
Mask shape: (16, 2, 128)
Segment IDs shape: (16, 2, 128)
Example token_ids[0,0,:5]: tf.Tensor([  101 25732  1024  1045  2052], shape=(5,), dtype=int32) tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int32)


In [15]:
def process_response(inputs, index):
    return {
        "token_ids": inputs["token_ids"][:, index, :],
        "padding_mask": inputs["padding_mask"][:, index, :],
        "segment_ids": inputs["segment_ids"][:, index, :]
    }
    
def build_bert_classifier():
    inputs = {
        "token_ids": tf.keras.Input(shape=(2, SEQUENCE_LENGTH), dtype=tf.int32, name="token_ids"),
        "padding_mask": tf.keras.Input(shape=(2, SEQUENCE_LENGTH), dtype=tf.int32, name="padding_mask"),
        "segment_ids": tf.keras.Input(shape=(2, SEQUENCE_LENGTH), dtype=tf.int32, name="segment_ids")
    }

    backbone = keras_hub.models.BertBackbone.from_preset(MODEL_NAME)

    emb_a = backbone(process_response(inputs, 0))["pooled_output"]
    emb_b = backbone(process_response(inputs, 1))["pooled_output"]

    embs = tf.keras.layers.Concatenate()([emb_a, emb_b])
    x = embs
    #x = tf.keras.layers.GlobalAveragePooling1D()(embs)
    #x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(3, activation="softmax", name="classifier")(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

model = build_bert_classifier()

In [16]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ padding_mask        │ (None, 2, 128)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ segment_ids         │ (None, 2, 128)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_ids           │ (None, 2, 128)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 128)       │          0 │ padding_mask[0][… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_3          │ (None, 128)       │          0 │ segment_ids[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 128)       │          0 │ token_ids[0][0]   │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_5          │ (None, 128)       │          0 │ padding_mask[0][… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_6          │ (None, 128)       │          0 │ segment_ids[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_4          │ (None, 128)       │          0 │ token_ids[0][0]   │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_backbone       │ [(None, 512),     │ 28,763,648 │ get_item_2[0][0], │
│ (BertBackbone)      │ (None, 128, 512)] │            │ get_item_3[0][0], │
│                     │                   │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_5[0][0], │
│                     │                   │            │ get_item_6[0][0], │
│                     │                   │            │ get_item_4[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1024)      │          0 │ bert_backbone[0]… │
│ (Concatenate)       │                   │            │ bert_backbone[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ classifier (Dense)  │ (None, 3)         │      3,075 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 28,766,723 (109.74 MB)

 Trainable params: 28,766,723 (109.74 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

model.compile(
    optimizer=tf.keras.optimizers.AdamW(LEARNING_RATE, weight_decay=0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy", "categorical_crossentropy"]
)

In [18]:
# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.weights.h5", save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=1)
]

# Train model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/5


I0000 00:00:1758200508.323025      69 service.cc:148] XLA service 0x7d885800c630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758200508.324454      69 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1758200508.324479      69 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1758200510.905935      69 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1758200532.502617      69 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3234/3234 ━━━━━━━━━━━━━━━━━━━━ 615s 174ms/step - accuracy: 0.3724 - categorical_crossentropy: 1.1094 - loss: 1.1121 - val_accuracy: 0.4410 - val_categorical_crossentropy: 1.0594 - val_loss: 1.0676 - learning_rate: 5.0000e-06
Epoch 2/5
3234/3234 ━━━━━━━━━━━━━━━━━━━━ 542s 167ms/step - accuracy: 0.4264 - categorical_crossentropy: 1.0641 - loss: 1.0714 - val_accuracy: 0.4476 - val_categorical_crossentropy: 1.0521 - val_loss: 1.0624 - learning_rate: 5.0000e-06
Epoch 3/5
3234/3234 ━━━━━━━━━━━━━━━━━━━━ 541s 167ms/step - accuracy: 0.4501 - categorical_crossentropy: 1.0474 - loss: 1.0575 - val_accuracy: 0.4553 - val_categorical_crossentropy: 1.0502 - val_loss: 1.0617 - learning_rate: 5.0000e-06
Epoch 4/5
3234/3234 ━━━━━━━━━━━━━━━━━━━━ 540s 167ms/step - accuracy: 0.4709 - categorical_crossentropy: 1.0321 - loss: 1.0448 - val_accuracy: 0.4483 - val_categorical_crossentropy: 1.0543 - val_loss: 1.0671 - learning_rate: 5.0000e-06
Epoch 5/5
3234/3234 ━━━━━━━━━━━━━━━━━━━━ 540s 167ms/step - accuracy: 0

In [19]:
probs = model.predict(test_ds)
preds = np.argmax(probs, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


In [20]:
output = pd.DataFrame({'id': test_df["id"],
                        'winner_model_a': probs[:, 0],
                        'winner_model_b': probs[:, 1],
                        'winner_tie': probs[:, 2]})
output.to_csv('submission.csv', index=False)
output

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.320571,0.467139,0.212290
1,211333,0.307244,0.376846,0.315910
2,1233961,0.357008,0.352556,0.290437
